In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
from math import radians, sin, cos, sqrt, asin
from backend.global_variables import DATA_PATH
load_dotenv()


def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on the Earth's surface."""
    R = 6371.0  # Earth's radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    return R * c

In [4]:
villes_df = pd.read_csv("/Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/data/calculated_travels/name-stade.csv")
localisation_stades_df = pd.read_csv("/Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/data/calculated_travels/localisation_stade.csv", index_col=0)

villes_df = villes_df.merge(localisation_stades_df[["Stadium", "latitude", "longitude"]], on="Stadium", how="left")

In [78]:
API_TOKEN = "eccc953e-c1f7-45f4-a828-bc319440fd3a"
url = f"https://api.sncf.com/v1/coverage/sncf/places"
places_ville = []

for idx_row, row in villes_df.iterrows():
    query = row["Location"]
    params = {"q": query}
    response = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
    data = response.json()
    for place in data["places"]:
        if place["id"].startswith("stop_area:"):
            if f"({row['Location']})" in place["name"]:
                latitude = float(place["stop_area"]["coord"]["lat"])
                longitude = float(place["stop_area"]["coord"]["lon"])
                places_ville.append(
                    {
                    "location": query,
                    "team_name": row["Team"],
                    "stadium_name": row["Stadium"],
                    "stop_area_id": place["id"],
                    "gare_name": place["name"],
                    "latitude": latitude,
                    "longitude": longitude,
                    "distance_to_stadium": haversine(float(row["latitude"]), float(row["longitude"]), latitude, longitude),
                    })

In [ ]:
pd.DataFrame(places_ville).to_csv(DATA_PATH + "gare_positions.csv", index=False)